In [ ]:
from scipy.optimize import differential_evolution
import joblib
import os
# we use this because our model (tree) is a stepwise not a smooth cont (calculus) fxn
# grid search would be faster for two variabless but we use this for antifragility (more var)
# our opt is finding the lowest possible altitude blindfolded on a hill
# it droops paratroopers randomly and checks their altitude
# kills everyone higher and keeps only the lower
# drops next gen of paratr below or around the lowest and as such

NB_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(NB_DIR)
model_path = os.path.join(PROJECT_ROOT, 'models')

def obj_fxn(x):

    # here x is a two element list [thrust, rpm]
    current_thrust = x[0]
    current_rpm = x[1]

    model = joblib.load(os.path.join(model_path, 'xgb_model.pkl'))
    rop = model.predict

    PRICE_PER_CUTTER = 1500 # $
    WEAR_FACTOR = 0.0005  # how to calc in real data?

    PRICE_GRID = 0.09
    PRICE_DIESEL = 0.35 # usd per KWh

    Power_KW = current_thrust * current_rpm / 100


    energy_cost_m = Power_KW * Energy_Price / ROP_m_hr
    cutter_wear_index = UCS_MPa**2 * current_rpm * WEAR_FACTOR

    cutter_cost_m = (cutter_wear_index * PRICE_PER_CUTTER) / ROP_m_hr
    Fixed_Cost_Hourly = 2000
    time_cost_m = Fixed_Cost_Hourly / ROP_m_hr
    total_cost = time_cost_m + cutter_cost_m + energy_cost_m
    return total_cost




bounds = [ ]        # bounds for rpm and thrust

In [ ]:
result = differential_evolution(
    obj_fxn, # what we need to optimize
    bounds,
    strategy='best1bin', #"Take the Best Guy's position, add a little bit of randomness based on how far apart the other guys are, and spawn a new guy there."
    # bin = binomial crossover, we could use rand1bin, for eng control best1bin is fine
    maxiter=20, # generations of trooper landing on hills
    popsize=10, # no of paratr in a gen
    tol=0.01 # stop when improvement < 1%
    seed=42 # its a randomized guess, we want it to guess in the same order for now
)